In [3]:
# @title Подключение к диску с данными
import os
from google.colab import drive
drive.mount('/content/drive')
!pip install polars
!pip install Dask
!pip install lightgbm

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.4/810.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 91.8 MB/s eta 0:00:00


In [1]:
# @title Шаг 3: Подготовка фич и предсказания на Test (по частям)

import polars as pl
import lightgbm as lgb
import gc
from datetime import datetime
import numpy as np

print("Начало инференса на Test", datetime.now().strftime("%Y-%m-%d %H:%M"))

DATA_PATH = "/content/drive/MyDrive/ml-vtb-data-fusion-strazh/data/"
MODEL_FILE = "/content/models/lgb_chunked_fixed_train_1-2_PR0.1420.txt"

# Загружаем модель
booster = lgb.Booster(model_file=MODEL_FILE)
print("Модель загружена.")


Начало инференса на Test 2026-02-18 19:54
Модель загружена.


In [2]:
def load_and_prepare_part(df):

    for col in cat_features_names:
        if col in df.columns:
            df = df.with_columns(pl.col(col).cast(pl.Utf8).cast(pl.Categorical).to_physical())


    feature_cols = [c for c in df.columns if c not in exclude_cols]

    # Получаем индексы категориальных признаков (нужны для lightgbm)
    cat_indices = []
    for i, col in enumerate(feature_cols):
        if col in cat_features_names:
            cat_indices.append(i)

    return df.select(feature_cols).to_numpy()

cat_features_names = [
    "event_type_nm",
    "channel_indicator_type",
    "channel_indicator_sub_type",
    "currency_iso_cd",
    "mcc_code",
    "pos_cd",
    "accept_language",
    "browser_language",
    "timezone",
    "operating_system_type",
    "device_system_version",
    "screen_size",
    "event_desc",
    "battery"
]
exclude_cols = ["customer_id", "event_id", "event_dttm", "date", "target"]

In [8]:
# @title Объединение истории (Pre-train + Train + Pre-test + Test)
# Периоды: Pre-train (3 части), Train (3 части), Pre-test (1 файл?), Test (1 файл)
# @title Объединение всей истории с приведением схемы (без OOM)

# Список всех файлов/частей
pretrain_files = [f"{DATA_PATH}pretrain_part_{i}.parquet" for i in [1,2,3]]
train_files    = [f"{DATA_PATH}train_part_{i}.parquet"    for i in [1,2,3]]
pretest_file   = f"{DATA_PATH}pretest.parquet"           # ← уточни имя файла
test_file      = f"{DATA_PATH}test.parquet"               # ← уточни имя файла

all_files = pretrain_files + train_files + [pretest_file, test_file]

# Список колонок, которые нужно привести к String (самый безопасный тип для кодов/MCC)
to_string_cols = [
    "event_type_nm",
    "channel_indicator_type",
    "channel_indicator_sub_type",
    "currency_iso_cd",
    "mcc_code",
    "pos_cd",
    "accept_language",
    "browser_language",
    "timezone",
    "operating_system_type",
    "device_system_version",
    "screen_size",
    "event_desc",
    "battery"
]

lazy_dfs = []

for file in all_files:
    df = pl.scan_parquet(file)

    # Приводим подозрительные колонки к String
    for col in to_string_cols:
        if col in df.columns:
            df = df.with_columns(pl.col(col).cast(pl.Utf8))

    # Парсим время (если ещё не сделано)
    if "event_dttm" in df.columns:
        df = df.with_columns(
            pl.col("event_dttm").str.to_datetime("%Y-%m-%d %H:%M:%S", strict=False)
        )

    lazy_dfs.append(df)

# Теперь concat лениво
big_history = pl.concat(lazy_dfs, how="vertical_relaxed")   # relaxed — игнорирует мелкие различия

print("Объединение завершено (лениво). Схема:")
print(big_history.schema)

# Если хочешь убедиться — собери небольшую выборку
# big_history.head(100).collect().write_parquet("debug_schema.parquet")

/tmp/ipython-input-1540426365.py:38: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if col in df.columns:
/tmp/ipython-input-1540426365.py:42: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "event_dttm" in df.columns:


Объединение завершено (лениво). Схема:
Schema({'customer_id': Int64, 'event_id': Int64, 'event_dttm': Datetime(time_unit='us', time_zone=None), 'event_type_nm': String, 'event_desc': String, 'channel_indicator_type': String, 'channel_indicator_sub_type': String, 'operaton_amt': Float64, 'currency_iso_cd': String, 'mcc_code': String, 'pos_cd': String, 'accept_language': String, 'browser_language': String, 'timezone': String, 'session_id': Int64, 'operating_system_type': String, 'battery': String, 'device_system_version': String, 'screen_size': String, 'developer_tools': String, 'phone_voip_call_state': Int32, 'web_rdp_connection': Int32, 'compromised': String})


/tmp/ipython-input-1540426365.py:53: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  print(big_history.schema)


In [9]:
for file in all_files:
    print(f"\nСхема файла: {file}")
    df = pl.scan_parquet(file).limit(1).collect()
    print(len(df.schema.items()))
    for col, dtype in df.schema.items():
        print(f"  {col:25} → {dtype}")


Схема файла: /content/drive/MyDrive/ml-vtb-data-fusion-strazh/data/pretrain_part_1.parquet
23
  customer_id               → Int64
  event_id                  → Int64
  event_dttm                → String
  event_type_nm             → Int32
  event_desc                → Int32
  channel_indicator_type    → Int32
  channel_indicator_sub_type → Int32
  operaton_amt              → Float64
  currency_iso_cd           → Int32
  mcc_code                  → String
  pos_cd                    → Int32
  accept_language           → String
  browser_language          → String
  timezone                  → Int32
  session_id                → Int32
  operating_system_type     → Int32
  battery                   → String
  device_system_version     → String
  screen_size               → String
  developer_tools           → String
  phone_voip_call_state     → Int32
  web_rdp_connection        → Int32
  compromised               → String

Схема файла: /content/drive/MyDrive/ml-vtb-data-fusion-strazh/da

In [10]:
# @title Feature Engineering на всей истории (по частям, чтобы не OOM)
# Разбиваем на 3 части по customer_id (как раньше)
customer_ids = big_history.select("customer_id").unique().collect()["customer_id"].to_list()
chunk_size = len(customer_ids) // 3
chunks = [customer_ids[i:i+chunk_size] for i in range(0, len(customer_ids), chunk_size)]

for chunk_idx, cust_chunk in enumerate(chunks, 1):
    print(f"Обработка чанка {chunk_idx}/3...")

    df_chunk = big_history.filter(pl.col("customer_id").is_in(cust_chunk))
    labels = pl.scan_parquet(f"{DATA_PATH}train_labels.parquet")
    df_chunk = df_chunk.join(
        labels.select(["event_id", "target"]),
        on="event_id",
        how="left"
        ).with_columns(
            pl.col("target").fill_null(0).cast(pl.Int32)
            )
    # 1. Базовые time-features
    df_chunk = df_chunk.with_columns([
        pl.col("event_dttm").dt.hour().alias("hour"),
        pl.col("event_dttm").dt.weekday().alias("dow"),
        pl.col("event_dttm").dt.date().alias("date"),
        (pl.col("operaton_amt").log1p()).alias("log_amt")
    ])

    # 2. Device risk
    df_chunk = df_chunk.with_columns([
        pl.col("compromised").cast(pl.Int32),
        pl.col("developer_tools").cast(pl.Int32),
        pl.col("web_rdp_connection").cast(pl.Int32),
        pl.col("phone_voip_call_state").cast(pl.Int32)
    ])
    df_chunk = df_chunk.with_columns(
        pl.sum_horizontal(["compromised", "developer_tools", "web_rdp_connection", "phone_voip_call_state"]).alias("device_risk")
    )

    # 3. Rolling features
    windows = ["1h", "24h", "7d"]
    for w in windows:
        df_chunk = df_chunk.sort(["customer_id", "event_dttm"])
        agg = df_chunk.rolling(
            index_column="event_dttm",
            period=w,
            group_by="customer_id",
            closed="left"
        ).agg([
            pl.col("event_id").count().alias(f"cnt_{w}"),
            pl.col("operaton_amt").sum().alias(f"sum_amt_{w}"),
            pl.col("operaton_amt").mean().alias(f"mean_amt_{w}"),
            pl.col("mcc_code").n_unique().alias(f"uniq_mcc_{w}"),
        ])
        df_chunk = df_chunk.join(agg, on=["customer_id", "event_dttm"], how="left")

    # 4. TE (expanding, как у тебя)
    global_target_mean = 0.0007  # пример, возьми из твоего расчёта (51k / 150M)
    C = 15
    cat_cols = ["mcc_code", "event_type_nm", "channel_indicator_type"]
    for col in cat_cols:
        df_chunk = df_chunk.sort(["customer_id", col, "event_dttm"])
        df_chunk = df_chunk.with_columns([
            pl.col("target").cum_sum().over(["customer_id", col]).alias("cum_target_all"),
            pl.int_range(0, pl.len()).over(["customer_id", col]).alias("cum_cnt_all")
        ])
        df_chunk = df_chunk.with_columns([
            (pl.col("cum_target_all") - pl.col("target")).alias("cum_target_prev"),
            pl.col("cum_cnt_all").alias("cum_cnt_prev")
        ])
        df_chunk = df_chunk.with_columns(
            ((pl.col("cum_target_prev").fill_null(0) + C * global_target_mean) /
             (pl.col("cum_cnt_prev").fill_null(0) + C)).alias(f"te_{col}_30d")
        )
        df_chunk = df_chunk.drop(["cum_target_all", "cum_cnt_all", "cum_target_prev", "cum_cnt_prev"])

    # Сохраняем чанк
    df_chunk.collect(engine="streaming").write_parquet(f"/content/test_features_chunk_{chunk_idx}.parquet")
    del df_chunk
    gc.collect()

Обработка чанка 1/3...
Обработка чанка 2/3...
Обработка чанка 3/3...
Обработка чанка 4/3...


In [ ]:
# @title Concat чанков и фильтр только Test операций

chunks = [pl.scan_parquet(f"/content/test_features_chunk_{i}.parquet") for i in [1,2,3,4]]
test_features = pl.concat(chunks, how="vertical").collect(engine="streaming")

start_date = pl.date(2025, 6, 1)
end_date = pl.date(2025, 8, 9)

# Если есть колонка 'date'
test_df = test_features.filter(
    (pl.col("date") >= start_date) & (pl.col("date") <= end_date)
)

# Фильтруем только Test (по event_id из sample_submit или по датам 2025-06-01 — 2025-08-09 + финальный день клиента)
# test_file_df = pl.read_parquet(f"{test_file}")
# test_ids = test_file_df["event_id"].to_list()
# test_df = test_features.filter(pl.col("event_id").is_in(test_ids))

del test_features
gc.collect()

In [15]:
# @title Predict

feature_cols = [c for c in test_df.columns if c not in exclude_cols]  # exclude_cols из твоего кода

X_test = load_and_prepare_part(test_df)
preds_test = booster.predict(X_test, num_iteration=booster.best_iteration)

# Добавляем predict в df
test_df = test_df.with_columns(pl.Series("predict", preds_test))

# Сохраняем для сабмита
test_df.select(["event_id", "predict"]).write_csv(f"{DATA_PATH}submission.csv")
print("Предсказания сохранены в submission.csv")

Предсказания сохранены в submission.csv


In [ ]:
# @title Predict 2

feature_cols = [c for c in test_df.columns if c not in exclude_cols]  # exclude_cols из твоего кода

X_test = test_df.select(feature_cols).to_numpy()
preds_test = booster.predict(X_test, num_iteration=booster.best_iteration)

# Добавляем predict в df
test_df = test_df.with_columns(pl.Series("predict", preds_test))

# Сохраняем для сабмита
test_df.select(["event_id", "predict"]).write_csv(f"{DATA_PATH}submission.csv")
print("Предсказания сохранены в submission.csv")

In [ ]:
test_file = "test.parquet"
df = pl.scan_parquet(f"{DATA_PATH}{test_file}")

    # 1. Базовые time-features
df = df.with_columns([
        pl.col("event_dttm").dt.hour().alias("hour"),
        pl.col("event_dttm").dt.weekday().alias("dow"),
        pl.col("event_dttm").dt.date().alias("date"),
        (pl.col("operaton_amt").log1p()).alias("log_amt")
    ])

    # 2. Device risk
df = df.with_columns([
        pl.col("compromised").cast(pl.Int32),
        pl.col("developer_tools").cast(pl.Int32),
        pl.col("web_rdp_connection").cast(pl.Int32),
        pl.col("phone_voip_call_state").cast(pl.Int32)
    ])
df = df.with_columns(
        pl.sum_horizontal(["compromised", "developer_tools", "web_rdp_connection", "phone_voip_call_state"]).alias("device_risk")
    )

    # 3. Rolling features
windows = ["1h", "24h", "7d"]
for w in windows:
        agg = df.rolling(
            index_column="event_dttm",
            period=w,
            group_by="customer_id",
            closed="left"
        ).agg([
            pl.col("event_id").count().alias(f"cnt_{w}"),
            pl.col("operaton_amt").sum().alias(f"sum_amt_{w}"),
            pl.col("operaton_amt").mean().alias(f"mean_amt_{w}"),
            pl.col("mcc_code").n_unique().alias(f"uniq_mcc_{w}"),
        ])
        df = df.join(agg, on=["customer_id", "event_dttm"], how="left")

    # 4. TE (expanding, как у тебя)
global_target_mean = 0.0007  # пример, возьми из твоего расчёта (51k / 150M)
C = 15
cat_cols = ["mcc_code", "event_type_nm", "channel_indicator_type"]
for col in cat_cols:
        df = df.sort(["customer_id", col, "event_dttm"])
        df = df.with_columns([
            pl.col("target").cum_sum().over(["customer_id", col]).alias("cum_target_all"),
            pl.int_range(0, pl.len()).over(["customer_id", col]).alias("cum_cnt_all")
        ])
        df = df.with_columns([
            (pl.col("cum_target_all") - pl.col("target")).alias("cum_target_prev"),
            pl.col("cum_cnt_all").alias("cum_cnt_prev")
        ])
        df = df.with_columns(
            ((pl.col("cum_target_prev").fill_null(0) + C * global_target_mean) /
             (pl.col("cum_cnt_prev").fill_null(0) + C)).alias(f"te_{col}_30d")
        )
        df = df.drop(["cum_target_all", "cum_cnt_all", "cum_target_prev", "cum_cnt_prev"])

    # Сохраняем чанк
df.collect(streaming=True).write_parquet(f"/content/test_features.parquet")
del df
gc.collect()